In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
from shapely.ops import transform

In [12]:
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString

def prep_reach_dist_with_radius(wel_lon, wel_lat, stream_sf, reach_id, stream_pt_spacing, radius, nseed=1):
    """
    Calculate distances only for stream reaches within a specified radius of influence.
    """
    np.random.seed(nseed)
    
    # Create a point for the well location
    well_point = Point(wel_lon, wel_lat)
    
    # Filter the stream reaches within the radius of influence
    stream_sf = stream_sf[stream_sf.geometry.distance(well_point) <= radius]
    print(f"Number of reaches within radius: {len(stream_sf)}")

    # Helper function to sample points or create midpoint for short geometries
    def sample_stream_points(geometry, spacing):
        if geometry.is_empty or not isinstance(geometry, LineString):
            return []  # Skip invalid or empty geometries
        length = geometry.length
        if length < spacing:  # For short lines, add midpoint
            return [geometry.interpolate(0.5 * length)]
        num_points = int(np.ceil(length / spacing))
        return [geometry.interpolate(i * spacing) for i in range(num_points)]
    
    # Generate points for filtered stream reaches
    points = []
    for _, row in stream_sf.iterrows():
        reach_points = sample_stream_points(row.geometry, stream_pt_spacing)
        for point in reach_points:
            points.append({
                'reach': row[reach_id],
                'lon': point.x,
                'lat': point.y
            })
        # Ensure at least one record for every ReachCode, even if no points are generated
        if not reach_points:
            points.append({
                'reach': row[reach_id],
                'lon': row.geometry.centroid.x,
                'lat': row.geometry.centroid.y
            })
    
    # Convert to GeoDataFrame and calculate distances
    points_gdf = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy([p['lon'] for p in points], [p['lat'] for p in points]))
    points_gdf['dist'] = np.sqrt((points_gdf['lon'] - wel_lon)**2 + (points_gdf['lat'] - wel_lat)**2)
    
    return points_gdf[['reach', 'dist', 'lat', 'lon']]

In [14]:
stream_sf = gpd.read_file('C:/Users/betebari/Documents/_Jupyter/StrmDepletr/shapefiles/MWC_NHD_flowlines_utm.shp')

In [18]:
# Define inputs
wel_lon = 522984
wel_lat = 4270707
radius = 700  # Radius of influence in the same units as the CRS (e.g., meters for UTM)

# Run the function
result = prep_reach_dist_with_radius(
    wel_lon=wel_lon,
    wel_lat=wel_lat,
    stream_sf=stream_sf,
    reach_id='ReachCode',
    stream_pt_spacing=5,
    radius=radius
)

# Verify results
print(result.head())
print(f"Number of unique reaches processed: {result['reach'].nunique()}")

Number of reaches within radius: 13
            reach        dist           lat            lon
0  18010110010890  417.233688  4.270436e+06  522666.802201
1  18010110010890  419.328316  4.270438e+06  522662.271767
2  18010110010890  421.471854  4.270440e+06  522657.741332
3  18010110010890  423.663558  4.270442e+06  522653.210898
4  18010110010890  425.902685  4.270444e+06  522648.680463
Number of unique reaches processed: 7


In [30]:
grouped = result.groupby('reach').size()
print(grouped)

reach
18010110000649    398
18010110003813     32
18010110003819    193
18010110003820     10
18010110003822     69
18010110007796    145
18010110010890    103
dtype: int64


In [28]:
result.to_csv('stream_distances_radius700.csv', index=False)